# Yahoo Finance Stock Screener Using Daily Data & Indicators

## Uses the following indicators
* SMA: https://www.investopedia.com/terms/s/sma.asp
* RSI: https://www.investopedia.com/terms/r/rsi.asp
* Bollinger Bands: https://www.investopedia.com/articles/technical/102201.asp & https://www.investopedia.com/trading/using-bollinger-bands-to-gauge-trends/

## Overview 
* Process will pull OHLC data from yahoo finance for a few tickers
* Based on the idicators Buy, Sell, Neutral signals will be generated for the day

### Get OHLC data from yahoo finance and turn into dataframe

In [1]:
# import libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Define the symbols and date range
symbols = ['MMM', 'AAPL', 'GOOGL']  # Example list of symbols
start_date = '2022-12-01'

# Get the current date
current_date = datetime.today()
end_date = (current_date - timedelta(days=1)).strftime('%Y-%m-%d')

# Create an empty list to store the results
results = []

# Loop over each symbol
for symbol in symbols:
    # Fetch OHLC data from Yahoo Finance
    data = yf.download(symbol, start=start_date, end=end_date)

    # Calculate SMA
    period = 20
    data['SMA'] = data['Close'].rolling(window=period, min_periods=20).mean()

    # Calculate RSI
    period = 14
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    relative_strength = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + relative_strength))

    # Calculate Bollinger Bands
    period = 20
    std_dev = data['Close'].rolling(window=period).std().shift(1)
    data['Upper Band'] = data['SMA'] + 2 * std_dev
    data['Middle Band'] = data['SMA']
    data['Lower Band'] = data['SMA'] - 2 * std_dev

    # Calculate Signal columns
    data['SMA Signal'] = ''
    data['RSI Signal'] = ''
    data['BB Signal'] = ''

    data.loc[data['SMA'] > data['Close'], 'SMA Signal'] = 'Sell'
    data.loc[data['SMA'] < data['Close'], 'SMA Signal'] = 'Buy'
    data.loc[data['SMA'] == data['Close'], 'SMA Signal'] = 'Neutral'

    data.loc[data['RSI'] > 70, 'RSI Signal'] = 'Sell'
    data.loc[data['RSI'] < 30, 'RSI Signal'] = 'Buy'
    data.loc[(data['RSI'] >= 30) & (data['RSI'] <= 70), 'RSI Signal'] = 'Neutral'

    data.loc[(data['Close'] < data['Lower Band']) & (data['RSI'] < 30), 'BB Signal'] = 'Buy'
    data.loc[(data['Close'] > data['Upper Band']) & (data['RSI'] > 70), 'BB Signal'] = 'Sell'
    data.loc[((data['Close'] >= data['Lower Band']) & (data['Close'] <= data['Middle Band'])) | ((data['Close'] <= data['Upper Band']) & (data['Close'] >= data['Middle Band'])), 'BB Signal'] = 'Neutral'

    # Calculate Buy, Sell, and Neutral Signal counts
    data['Buy Signal'] = (data['SMA Signal'] == 'Buy').astype(int) + (data['RSI Signal'] == 'Buy').astype(int) + (data['BB Signal'] == 'Buy').astype(int)
    data['Sell Signal'] = (data['SMA Signal'] == 'Sell').astype(int) + (data['RSI Signal'] == 'Sell').astype(int) + (data['BB Signal'] == 'Sell').astype(int)
    data['Neutral Signal'] = (data['SMA Signal'] == 'Neutral').astype(int) + (data['RSI Signal'] == 'Neutral').astype(int) + (data['BB Signal'] == 'Neutral').astype(int)

    # Add the symbol and its data to the results list
    results.append({'Symbol': symbol, 'Data': data})

# Convert results to dataframes
dataframes = []
for result in results:
    symbol = result['Symbol']
    data = result['Data']
    dataframe = pd.DataFrame(data)
    dataframe['Symbol'] = symbol
    dataframes.append(dataframe)

# Combine all dataframes into a single dataframe
combined_dataframe = pd.concat(dataframes)
combined_dataframe.head(35)

# Sort by Symbol and then Date descending
# combined_dataframe.sort_values(by=['Symbol', 'Date'], ascending=[True, False], inplace=True)

# Display the combined dataframe
# combined_dataframe.head()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,SMA,RSI,Upper Band,Middle Band,Lower Band,SMA Signal,RSI Signal,BB Signal,Buy Signal,Sell Signal,Neutral Signal,Symbol
Date,,,,,,,,,,,,,,,,,,
2022-12-01,127.300003,128.869995,125.309998,125.989998,122.481842,2638800,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-02,124.760002,127.250000,124.239998,126.989998,123.453995,1821900,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-05,125.070000,125.459999,124.269997,124.629997,121.159706,2143100,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-06,125.150002,125.730003,123.309998,124.580002,121.111107,1919500,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-07,124.330002,127.309998,124.099998,126.349998,122.831810,2701500,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-08,127.000000,129.440002,125.449997,126.000000,122.491562,3785300,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-09,126.779999,127.529999,125.669998,125.760002,122.258247,2334900,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-12,126.080002,126.889999,124.199997,126.849998,123.317894,3877800,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM
2022-12-13,129.449997,130.020004,126.470001,127.290001,123.745644,3256900,NaN,NaN,NaN,NaN,NaN,,,,0,0,0,MMM


### Take the above data frame and show the 5 most recent days per symbol

In [3]:
# Sort the combined_dataframe by 'Symbol' and 'Date'
sorted_dataframe = combined_dataframe.sort_values(by=['Symbol', 'Date'], ascending=[True, False])

# Extract the most recent 5 days of data per ticker
grouped_dataframe = sorted_dataframe.groupby('Symbol').head(5)

print("Most recent 5 days of data per ticker:")
grouped_dataframe


Most recent 5 days of data per ticker:


,Open,High,Low,Close,Adj Close,Volume,SMA,RSI,Upper Band,Middle Band,Lower Band,SMA Signal,RSI Signal,BB Signal,Buy Signal,Sell Signal,Neutral Signal,Symbol
Date,,,,,,,,,,,,,,,,,,
2023-07-24,193.410004,194.910004,192.250000,192.750000,192.750000,45377800,191.037501,50.860513,196.274555,191.037501,185.800446,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-21,194.100006,194.970001,191.229996,191.940002,191.940002,71917800,190.734000,44.216534,196.192845,190.734000,185.275156,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-20,195.089996,196.470001,192.500000,193.130005,193.130005,59581200,190.487000,58.534245,196.521234,190.487000,184.452767,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-19,193.100006,198.229996,192.649994,195.100006,195.100006,80507300,190.028500,65.306126,195.963795,190.028500,184.093205,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-18,193.350006,194.330002,192.419998,193.729996,193.729996,48353800,189.524000,64.976217,195.452651,189.524000,183.595348,Buy,Neutral,Neutral,1,0,2,AAPL
2023-07-24,121.660004,123.000000,120.980003,121.529999,121.529999,29686100,120.527999,53.303601,125.580984,120.527999,115.475015,Buy,Neutral,Neutral,1,0,2,GOOGL
2023-07-21,120.620003,120.989998,118.730003,120.019997,120.019997,72937900,120.568499,50.684931,125.742392,120.568499,115.394606,Sell,Neutral,Neutral,0,1,2,GOOGL
2023-07-20,121.419998,124.089996,118.220001,119.199997,119.199997,37906800,120.725000,50.216073,125.850117,120.725000,115.599883,Sell,Neutral,Neutral,0,1,2,GOOGL
2023-07-19,124.599998,125.180000,121.800003,122.029999,122.029999,37224000,120.792500,54.324446,125.993782,120.792500,115.591218,Buy,Neutral,Neutral,1,0,2,GOOGL
